In [5]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.7 GB 9.3 MB/s eta 0:04:54
     ---------------------------------------- 0.0/2.7 GB 24.5 MB/s eta 0:01:51
     ---------------------------------------- 0.0/2.7 GB 28.0 MB/s eta 0:01:38
     ---------------------------------------- 0.0/2.7 GB 27.8 MB/s eta 0:01:38
     ---------------------------------------- 0.0/2.7 GB 31.5 MB/s eta 0:01:27
     ---------------------------------------- 0.0/2.7 GB 32.3 MB/s eta 0:01:24
     ---------------------------------------- 0.0/2.7 GB 30.4 MB/s eta 0:01:30
     ---------------------------------------- 0.0/2.7 GB 34.4 MB/s eta 0:01:19
     ---------------------------------------- 0.0/2.7 GB 34.4 MB/s eta 0:01:19
     ---------------------------------------- 0.0/2.7 GB 36.4 MB/s eta 0:01:15
     ---------------------------------------- 0.0/2.7 GB 40.9 MB/s eta 0:01:07
  

In [6]:
!pip install transformers requests beautifulSoup4 pandas numpy

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/1a/d1/3bba59606141ae808017f6fde91453882f931957f125009417b87a281067/transformers-4.34.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/121.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/121.5 kB ? eta -:--:--
     --- ------------------------------------ 10.2/121.5 kB ? eta -:--:--
     --------- --------------------------- 30.7/121.5 kB 325.1 kB/s eta 0:00:01
     ------------ ------------------------ 41.0/121.5 kB 279.3 kB/s eta 0:00:01
     ---------------------------- -------- 92.2/121.5 kB 521.8 kB/s eta 0:00:01
     ------------------------------------ 121.5/121.5 kB 592.1 kB/s eta 0:00:00
     ---------------------------------------- 0.0/143.0 kB ? eta -:--:--
     -------------------------------------- 143.0/143.0 kB 4.3 MB/s eta 0:00:00
  Obtaining dependency information for regex!=2019.12.17 from https://files.pythonhosted.org/pack

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re 

In [9]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

C:\Users\Kate\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Kate\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [30]:
tokens = tokenizer.encode("i love this product its really shit useless", return_tensors='pt')

In [31]:
result = model(tokens)
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.8082,  0.7698, -0.7701, -1.9372, -0.5245]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [32]:
int(torch.argmax(result.logits))+1

1

**Collect Reviews**

In [37]:
r = requests.get("https://www.yelp.com/biz/stella-mares-santa-barbara-2")
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [38]:
reviews

["Stella Mare's continues to be a 5 star worthy restaurant in my book. Their service is excellent, the food is incredible, the prices are very reasonable for the area and the quality of food, and you can't beat the ambiance. They also take reservations!I cake here twice recently with my mother and grandmother and we had fantastic meals. The bread basket at Sunday brunch with the mini carrot muffins was so good! Even the butter with the rosemary was delicious. We inquired as to whether they sell them a la carte as we wanted to buy some to take with us for the road and they said they dont sell them but the waitress packed us a to go box with 7 of them!! The duck burger continues to be one of my favorites. The croque madame was fantastic. My grandmother ordered the Coq au vin and loved it - the sweet potato mash in it was my favorite from what I sampled.  I just love everything about this restaurant and will continue to return whenever I pass through SB.",
 'I have been here a few times, 

In [41]:
results[0].text

"Stella Mare's continues to be a 5 star worthy restaurant in my book. Their service is excellent, the food is incredible, the prices are very reasonable for the area and the quality of food, and you can't beat the ambiance. They also take reservations!I cake here twice recently with my mother and grandmother and we had fantastic meals. The bread basket at Sunday brunch with the mini carrot muffins was so good! Even the butter with the rosemary was delicious. We inquired as to whether they sell them a la carte as we wanted to buy some to take with us for the road and they said they dont sell them but the waitress packed us a to go box with 7 of them!! The duck burger continues to be one of my favorites. The croque madame was fantastic. My grandmother ordered the Coq au vin and loved it - the sweet potato mash in it was my favorite from what I sampled.  I just love everything about this restaurant and will continue to return whenever I pass through SB."

**Load Reviews into DataFrame**

In [42]:
import numpy as np
import pandas as pd

In [43]:
df=pd.DataFrame(np.array(reviews), columns=['reviews'])

In [45]:
df.tail()

,reviews
6,Stella Mare's setting is the historical W. H. ...
7,Stella Mare's is a super cute place to dine at...
8,Gorgeous environment - loved the comfort and e...
9,So delicious. Came for a dinner and was so sur...
10,Fantastic! We went for a wedding lunch and eve...


In [46]:
df['reviews'].iloc[0]

"Stella Mare's continues to be a 5 star worthy restaurant in my book. Their service is excellent, the food is incredible, the prices are very reasonable for the area and the quality of food, and you can't beat the ambiance. They also take reservations!I cake here twice recently with my mother and grandmother and we had fantastic meals. The bread basket at Sunday brunch with the mini carrot muffins was so good! Even the butter with the rosemary was delicious. We inquired as to whether they sell them a la carte as we wanted to buy some to take with us for the road and they said they dont sell them but the waitress packed us a to go box with 7 of them!! The duck burger continues to be one of my favorites. The croque madame was fantastic. My grandmother ordered the Coq au vin and loved it - the sweet potato mash in it was my favorite from what I sampled.  I just love everything about this restaurant and will continue to return whenever I pass through SB."

In [47]:
def sent_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1


In [49]:
df['sentiment'] = df['reviews'].apply(lambda x: sent_score(x))

In [50]:
df

,reviews,sentiment
0,Stella Mare's continues to be a 5 star worthy ...,5
1,"I have been here a few times, the first two ti...",5
2,We originally made a reservation for Sunday b...,4
3,"I'm very happy with this place.Excellent food,...",5
4,Very beautiful local French place! We made a r...,4
5,"Frog legs, escargot, and roasted duck were all...",4
6,Stella Mare's setting is the historical W. H. ...,4
7,Stella Mare's is a super cute place to dine at...,3
8,Gorgeous environment - loved the comfort and e...,5
9,So delicious. Came for a dinner and was so sur...,5


In [51]:
df['reviews'].iloc[7]

"Stella Mare's is a super cute place to dine at! I decided to be adventurous and try snails (Escargots persillade) for the first time, and I absolutely loved it! The snails were cooked to perfection, with a smooth texture and a strong pesto flavor. However, I do wish the flavors were even stronger. The earthy aroma was a great touch though.The French onion soup was another standout dish. The distinct onion flavor was spot on, and the cheese added a layer of complexity that made it all the more delicious. I especially enjoyed the bread in the soup, which added another texture to the mix.The Confit de canard was pretty good too. The crispy texture on the outside was a nice contrast to the tender inside, and the sweetness of the cherries complemented the saltiness of the duck perfectly. For the braised lamb shank, I liked that the meat was very tender and that the tendon was chewy. However, it came with grits and I felt like it would've been better with mashed potatoes instead.Unfortunate